<h1>Retired People and “Retired Places”: Italy, Data Preparation<span class="tocSkip"></span></h1>
<div class="toc">
  <ul class="toc-item">
    <li>
      <span>
        <a href="#Project-description-and-data-sources" data-toc-modified-id="Project-description-and-data-sources-1">
          <span class="toc-item-num">1&nbsp;&nbsp;</span>Project description and data sources
        </a>
      </span>
    </li>
    <li>
      <span>
        <a href="#Loading-raw-tables" data-toc-modified-id="Loading-raw-tables-2">
          <span class="toc-item-num">2&nbsp;&nbsp;</span>Loading raw tables
        </a>
      </span>
    </li>
    <li>
      <span>
        <a href="#Data-cleaning-and-prepocessing" data-toc-modified-id="Data-cleaning-and-prepocessing-3">
          <span class="toc-item-num">3&nbsp;&nbsp;</span>Data cleaning and prepocessing
        </a>
      </span>
    </li>
    <li>
      <span>
        <a href="#Computing-demographic-and-housing-indicators" data-toc-modified-id="Computing-demographic-and-housing-indicators-4">
          <span class="toc-item-num">4&nbsp;&nbsp;</span>Computing demographic and housing indicators
        </a>
      </span>
    </li>
    <li>
      <span>
        <a href="#Building-the-2x2-categories" data-toc-modified-id="Building-the-2x2-categories-5">
          <span class="toc-item-num">5&nbsp;&nbsp;</span>Building the 2×2 categories
        </a>
      </span>
    </li>
    <li>
      <span>
        <a href="#Merging-with-geometries-and-saving-outputs" data-toc-modified-id="Merging-with-geometries-and-saving-outputs-6">
          <span class="toc-item-num">6&nbsp;&nbsp;</span>Merging with geometries and saving outputs
        </a>
      </span>
    </li>
  </ul>
</div>


## Loading-raw-tables <a id="Loading-raw-tables"></a>

In [1]:
import pandas as pd
import geopandas as gpd
from pathlib import Path

print('Libraries are downloaded')

Libraries are downloaded


In [2]:
PROJECT_ROOT = Path("..").resolve().parent
RAW = PROJECT_ROOT / "data" / "raw"

RAW

PosixPath('/Users/eugenia/Desktop/Open Access/project/retired_places/data/raw')

In [3]:
pop_reg_it = pd.read_csv(
    RAW / "popolazione_2025_it_regioni.csv",  
    sep=";",        
    skiprows=1,    
    encoding="utf-8"
)
display(pop_reg_it.head())

,Codice regione,Regione,Età,Totale maschi,Totale femmine,Totale
0,13,Abruzzo,0.0,3842.0,3577.0,7419.0
1,13,Abruzzo,1.0,4010.0,3653.0,7663.0
2,13,Abruzzo,2.0,4260.0,3873.0,8133.0
3,13,Abruzzo,3.0,4298.0,4163.0,8461.0
4,13,Abruzzo,4.0,4456.0,4163.0,8619.0


In [4]:
# 1) Load raw Excel WITHOUT headers to inspect structure
homes_raw = pd.read_excel(
    RAW / "abitazioni_occupate_non_occupate_reg.xlsx",
    sheet_name=0,
    header=None
)

display(homes_raw.head(15))


,0,1,2,3
0,Abitazioni occupate e non occupate - comuni,NaN,NaN,NaN
1,Frequenza: Annuale,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,Anno,2021,2021,2021
4,Indicatore,Abitazioni occupate al 31 dicembre,Abitazioni non occupate al 31 dicembre,Abitazioni al 31 dicembre
5,Anno base,,,
6,Unità di misura,,,
7,Unità di moltiplicazione,,,
8,Territorio,,,
9,Piemonte,1964108,827768,2791876


## Data cleaning and prepocessing <a id="Data-cleaning-and-prepocessing"></a>

### Adjusting columns names 

In [5]:
# Keep only the data rows starting from the first region 
homes_it = homes_raw.iloc[9:, :4].copy()

# Assign clean column names
homes_it.columns = ["region", "homes_occupied", "homes_unoccupied", "homes_total"]

# Reset index
homes_it = homes_it.reset_index(drop=True)

# Final check
display(homes_it.head())


,region,homes_occupied,homes_unoccupied,homes_total
0,Piemonte,1964108,827768,2791876
1,Valle d'Aosta / Vallée d'Aoste,59616,75948,135564
2,Liguria,746686,431321,1178007
3,Lombardia,4415364,1184728,5600092
4,Trentino Alto Adige / Südtirol,463305,219888,683193


In [7]:
pop_reg_it.columns = ["region_code", "region", "age", "pop_male", "pop_female", "pop_total"]

pop_reg_it.head()

,region_code,region,age,pop_male,pop_female,pop_total
0,13,Abruzzo,0.0,3842.0,3577.0,7419.0
1,13,Abruzzo,1.0,4010.0,3653.0,7663.0
2,13,Abruzzo,2.0,4260.0,3873.0,8133.0
3,13,Abruzzo,3.0,4298.0,4163.0,8461.0
4,13,Abruzzo,4.0,4456.0,4163.0,8619.0


### Checking data types

In [11]:
print("=== homes_it: dtypes ===")
print(homes_it.dtypes)
print("\n")

print("=== pop_reg_it: dtypes ===")
print(pop_reg_it.dtypes)


=== homes_it: dtypes ===
region              object
homes_occupied      object
homes_unoccupied    object
homes_total         object
dtype: object


=== pop_reg_it: dtypes ===
region_code     object
region          object
age            float64
pop_male       float64
pop_female     float64
pop_total      float64
dtype: object


In [19]:
# Housing counts in homes_it  → Int64
housing_numeric_cols = ["homes_occupied", "homes_unoccupied", "homes_total"]

for col in housing_numeric_cols:
    homes_it[col] = (
        pd.to_numeric(homes_it[col], errors="coerce")  # just in case
        .round()
        .astype("Int64")                              # nullable integer
    )

# Population counts in pop_reg_it → Int64
population_numeric_cols = ["region_code", "age", "pop_male", "pop_female", "pop_total"]

for col in population_numeric_cols:
    pop_reg_it[col] = (
        pd.to_numeric(pop_reg_it[col], errors="coerce")
        .round()
        .astype("Int64")
    )

print("=== homes_it: dtypes after conversion ===")
print(homes_it.dtypes)
print("\n")
print("=== pop_reg_it: dtypes after conversion ===")
print(pop_reg_it.dtypes)

=== homes_it: dtypes after conversion ===
region              object
homes_occupied       Int64
homes_unoccupied     Int64
homes_total          Int64
dtype: object


=== pop_reg_it: dtypes after conversion ===
region_code     Int64
region         object
age             Int64
pop_male        Int64
pop_female      Int64
pop_total       Int64
dtype: object
